# 问题三的单独求解
暴力手算

In [1055]:
# 导入包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math 
from scipy.optimize import fsolve

# 问题一

## 基本功能函数

### 计算向量的模

In [1056]:
# 定义向量
v = np.array([3, 4])

# 计算向量的模
magnitude = np.linalg.norm(v)

print(f"向量的模为: {magnitude}")

向量的模为: 5.0


### 单位化向量

In [1057]:
def compute_unit_vector(vx, vy):
    """
    计算单位向量
    输入: vx -  x 分量
          vy -  y 分量
    输出: 单位向量 (vx_n, vy_n)
    """
    # 计算速度的大小
    magnitude = math.sqrt(vx**2 + vy**2)
    
    # 计算单位向量
    vx_n = vx / magnitude
    vy_n = vy / magnitude
    
    return (vx_n, vy_n)

### 由 $\theta$ 求解极径r的值
螺线方程为： $r=8.8 - \frac{0.55}{2\pi}\theta $

In [1058]:
def compute_radius(θ):
    """
    计算螺线方程 r = 8.8 + (0.55 / (2 * pi)) * theta 的极径 r
    输入: theta - 角度（弧度制）
    输出: 对应的极径 r
    """
    global p
    # 螺线方程
    r = 8.8 + ( p / (2 * math.pi)) * θ
    return r

### 由 $\theta$ 计算点坐标

In [1059]:
def compute_coordinates(θ):
    """
    计算点的坐标 (r cos(θ), r sin(θ))
    输入: theta - 角度（弧度制）
    输出: 对应的坐标 (x, y)
    """
    # 计算极径
    r = compute_radius(θ)
    
    # 计算坐标
    x = r*math.cos(θ)
    y = r*math.sin(θ)
    return (x, y)


### 求解龙头前把手的位置
$\overrightarrow {v_1}=(r_1 cos \theta_1, -r_1 sin \theta_1)$

In [1060]:
def compute_hand_position(θ):
    """
    计算在时间 t 时刻龙头前把手的位置
    输入: theta - 角度（弧度制）
    输出: 对应的坐标 (x, y)
    """
    
    return compute_coordinates(θ)

##  求解位置


### 求出龙头在300s内的所有 $\theta$ 值
从0秒到300秒，共301个时间点

#### 计算函数

In [1061]:
from scipy.optimize import fsolve

def f(x):
    """
    计算函数 f(x)
    输入: x - 自变量
    输出: f(x) 的值
    """
    return 0.5 * x * np.sqrt(x**2 + 1) + 0.5 * np.log(np.abs(x + np.sqrt(x**2 + 1)))

def g(x, t):
    """
    计算函数 g(x) - t
    输入: x - 自变量
           t - 目标值
    输出: g(x) - t 的值
    """
    global p
    term1 = f(x + 32 * np.pi)
    term2 = f(32 * np.pi)
    return ( p / (2 * np.pi)) * (term2 - term1) - t

def solve_θ_of_dragonhead(t_value):
    """
    解方程 g(x) = t 的 x 值
    输入: t_value - 目标值 t
    输出: 对应的 x 值
    """
    # 初始猜测值为0
    initial_guess = np.radians(0)

    try:
        # 使用 fsolve 求解 g(x) - t = 0 的方程
        x_value = fsolve(g, x0=initial_guess, args=(t_value))
        return x_value[0]
    except Exception as e:
        print(f"Failed to solve: {e}")
        return None


#### 计算并存储龙头的 θ 值

In [1062]:
# 初始化时间数组
time_array = np.arange(301)  # 从 0 到 300 的整数数组

# 计算每个时间点的角度值（弧度制）
theta_head_value = np.array([solve_θ_of_dragonhead(t) for t in time_array])

In [1063]:
# 输出300s 内的所有龙的 θ 值
print("前几个 θ 值 (弧度制):")
print(theta_head_value)  


前几个 θ 值 (弧度制):
[  0.          -0.15778503  -0.31581847  -0.47410148  -0.63263526
  -0.79142099  -0.9504599   -1.10975318  -1.26930207  -1.42910779
  -1.5891716   -1.74949475  -1.9100785   -2.07092413  -2.23203292
  -2.39340617  -2.55504518  -2.71695128  -2.87912579  -3.04157004
  -3.2042854   -3.36727322  -3.53053488  -3.69407175  -3.85788525
  -4.02197677  -4.18634773  -4.35099957  -4.51593374  -4.68115169
  -4.84665488  -5.01244481  -5.17852297  -5.34489086  -5.51155001
  -5.67850195  -5.84574824  -6.01329043  -6.1811301   -6.34926884
  -6.51770825  -6.68644996  -6.8554956   -7.02484681  -7.19450525
  -7.36447262  -7.53475059  -7.70534088  -7.87624522  -8.04746534
  -8.219003    -8.39085997  -8.56303805  -8.73553904  -8.90836477
  -9.08151707  -9.2549978   -9.42880884  -9.60295209  -9.77742945
  -9.95224286 -10.12739426 -10.30288562 -10.47871894 -10.65489621
 -10.83141946 -11.00829074 -11.18551212 -11.36308568 -11.54101354
 -11.71929781 -11.89794065 -12.07694424 -12.25631076 -12.4360

### 求出对应于每个 $\theta$ 的 所对应的所有龙身的 $\theta$ 值

#### 计算函数

In [1064]:
from scipy.optimize import fsolve

def g_function(θ2, θ1, b):
    """
    计算方程 g(θ2) - b^2 的值，用于求解 θ2
    输入: theta2 - 待求解的角度 θ2（弧度制）
           theta1 - 已知角度 θ1（弧度制）
           b - 目标值 b
    输出: g(θ2) - a^2 的值
    """
    r1 = compute_radius(θ1)
    r2 = compute_radius(θ2)
    # g(θ2) 的表达式
    g_θ2 = r2**2 - 2 * r1 * r2 * np.cos(θ2 - θ1) + r1**2
    return g_θ2 - b**2

def compute_theta_next(θ1, b):
    """
    使用数值方法计算 θ2，使得 g(θ2) = b^2 且 θ2 - θ1 ∈ (0, 2π)
    输入: theta1 - 已知角度 θ1（弧度制）
           b - 目标值 b
    输出: 满足条件的 θ2 的值
    """
    # 定义多个初始猜测值，用于 fsolve 寻找不同的解
    initial_guesses = [ θ1+0.2, θ1+0.4, θ1+0.7, θ1+0.9]  
    solutions = []

    # 对每个初始猜测值进行求解
    for guess in initial_guesses:
        θ2_solution = fsolve(g_function, guess, args=(θ1, b))
        θ2 = θ2_solution[0]

        # 检查解是否满足  θ2 - θ1 ∈ (-2π, 0)
        if  0 < (θ2 - θ1) < 2*np.pi:
            solutions.append(θ2)

    # 返回满足条件的最小解
    if solutions:
        unique_solutions = np.unique(solutions)  # 去重
        return min(solutions)
    else:
        return None  # 如果没有满足条件的解，返回 None

示例调用

#### 计算每个时间点的所有龙身的 θ 值

In [1065]:
length_of_dragon_head = 2.86 # 龙头的长度 286cm=2.86m
distance_of_dragon_body= 1.65 # 龙身上两个孔的距离 165cm=1.65m

# 创建二维数组来存储每个时间点的所有龙身的 θ 值
theta_body_values_of_each_time = np.empty((301, 223))  # 301 行，222 列；301个时间点，221个龙身，1个龙尾

# 计算每个时间点的所有龙身的 θ 值
for t in range(301):
    temp =  theta_head_value[t] # 当前时间内点的龙头对应的θ值
    for i in range(223):
        if i == 0:
            theta_body_values_of_each_time[t, i] = compute_theta_next(temp, length_of_dragon_head) # 当前时间点的龙身每个点的θ值
            temp = theta_body_values_of_each_time[t, i] # 更新当前时间点的龙身对应的θ值
        else:
            theta_body_values_of_each_time[t, i] = compute_theta_next(temp, distance_of_dragon_body) # 当前时间点的龙身每个点的θ值
            temp = theta_body_values_of_each_time[t, i] # 更新当前时间点的龙身对应的θ值
            

In [1066]:
# 定义保存数组到 CSV 文件的函数
def save_array_to_csv(array, filename):
    """
    将二维数组保存到 CSV 文件
    输入:
        array - 要保存的数组
        filename - 保存的文件名
    """
    # 使用 numpy.savetxt 将数组保存到 CSV 文件中
    np.savetxt(filename, array, fmt='%.6f', delimiter=',')  # fmt 控制浮点数精度，delimiter 控制分隔符为逗号

# 调用函数将数组保存到 CSV 文件中
save_array_to_csv(theta_body_values_of_each_time, './Q1/theta_body_values_Q1.csv')

print("二维数组已保存到 ./Q1/theta_body_values_Q1.csv 文件中。")


二维数组已保存到 ./Q1/theta_body_values_Q1.csv 文件中。


In [1067]:
# 将所有时刻的所有点的位置信息存储到同一变量中，301行，223列，第一列为龙头，其余为龙身
theta_values_of_each_time = np.hstack((theta_head_value.reshape(-1, 1), theta_body_values_of_each_time))

## 求解速度

### 计算点的速度

In [1068]:
def compute_radius_derivative(θ):
    """
    计算 r(θ) = 8.8 - (0.55 / (2π)) * θ 的导数 
    dr/dθ= -0.55 / (2π)
    输出: 导数表达式的值
    """
    global p
    # 定义 r(theta) 的方程
    dr =   (p / (2 * np.pi)) 
    
    return dr

In [1069]:
def compute_velocity(θ):
    """
    计算点的速度
    """
    # 计算r的值
    r = compute_radius(θ)
    # 计算r的导数值
    dr = compute_radius_derivative(θ)

    # 计算点的速度
    vx = - r * np.sin(θ) + dr * np.cos(θ)
    vy =   r * np.cos(θ) + dr * np.sin(θ)
    
    return (vx, vy)

### 计算速度的单位方向向量
$v_n$ 表示速度的单位向量
$v_n =({v_x}_n,{v_y}_n)$

### 板凳的方向向量
$\vec{l}=(r_{n+1}\cos{\theta_{n+1}}-r_n\cos{\theta_n},r_{n+1}\sin{\theta_{n+1}}-r_n\sin(\theta_n))=(\cos{\alpha_n},sin{\alpha_n})$

In [1070]:
def compute_bench_direction_vector(θ_n, θ_n1):
    """
    计算板凳的方向向量
    输入: 
        r_n, theta_n - 当前点的极径和角度（弧度制）
        r_n1, theta_n1 - 下一个点的极径和角度（弧度制）
    输出: 
        向量 (lx, ly) 
    """
    # r_n 和 r_n1
    r_n = compute_radius(θ_n)
    r_n1 = compute_radius(θ_n1)

    # 计算方向向量的 x 和 y 分量
    lx = r_n * math.cos(θ_n) - r_n1 * math.cos(θ_n1) 
    ly = r_n * math.sin(θ_n) - r_n1 * math.sin(θ_n1)  
    
    return lx, ly



In [1071]:
# 示例调用
theta_n = math.radians(30)  # 当前点的角度
theta_n1 = math.radians(45) # 下一个点的角度

vector = compute_bench_direction_vector(theta_n, theta_n1)
print(f"板凳的方向向量为: {vector}")


unit_vector_l = compute_unit_vector(*vector)
print(f"板凳的方向单位向量为: {unit_vector_l}")

板凳的方向向量为: (1.392054443691995, -1.8410601487827458)
板凳的方向单位向量为: (0.6031174697777792, -0.7976523789526672)


### 计算并存储t时刻龙头的前把手的速度的大小


In [1072]:
#  speed_of_head 表示每个时刻的龙头的速度大小
speed_of_head= 1

### 计算并存储t时刻龙头的所有前把手的速度的大小
$u_n$ 表示速度的大小
$u_{n+1} = \frac{\vec{v_n}\cdot \vec l_n}{\overrightarrow{v_{n+1}} \cdot \vec l_n} \cdot u_n$

#### 计算函数

In [1073]:
def compute_velocity_next(θ_n, θ_n1, u_n):
    """
    计算速度大小 u_{n+1}
    
    参数：
    u_n: 当前点速度的大小
    v_n: 当前点的速度方向向量
    v_n1: 下一点的速度方向向量
    l: 参考方向的单位向量
    
    返回：
    u_{n+1}: 下一点的速度大小
    """

    # 计算速度方向向量
    v_n = compute_unit_vector(*compute_velocity(θ_n))
    v_n1 = compute_unit_vector(*compute_velocity(θ_n1))

    # 计算板凳的方向向量
    l_n_x, l_n_y = compute_bench_direction_vector(θ_n, θ_n1)
    l_n = np.array([l_n_x, l_n_y])
    
    # 计算点乘
    numerator = np.dot(v_n, l_n) # 计算分子
    denominator = np.dot(v_n1, l_n)   # 计算分母
    
    # 确保分母不为零
    if denominator == 0:
        raise ValueError("Denominator is zero, cannot divide.")
    
    # 更新速度大小
    u_n1 = (numerator / denominator) * u_n

    return u_n1


In [1074]:
# 建立一个空的数组来存储每个时间点龙身每个点的速度
velocity_body_values_of_each_time = np.empty((301, 224))  # 301 行，223 列；301个时间点，一个龙头，221个龙身，1个龙尾，最后一列为龙尾后把手

for t in range(301):
    temp = speed_of_head # 计算当前时间点的龙头速度大小
    velocity_body_values_of_each_time[t, 0] = temp # u0 存放龙头的速度大小
    for i in range(0,223):
        velocity_body_values_of_each_time[t, i+1] = compute_velocity_next(theta_values_of_each_time[t,i], theta_values_of_each_time[t,i+1], temp)
        temp = velocity_body_values_of_each_time[t, i+1]

In [1075]:
# 将矩阵保存到 txt 文件中
output_file_path = "./Q1/velocity_body_values.txt"  # 文件路径，可以根据需要修改

# 保存矩阵到 txt 文件，格式化为小数点后两位
np.savetxt(output_file_path, velocity_body_values_of_each_time, fmt='%.6f', delimiter=',', header='Velocity values for each time step')

print(f"速度矩阵已成功保存到 {output_file_path}")

速度矩阵已成功保存到 ./Q1/velocity_body_values.txt


## 输出结果整理


### 位置信息

In [1076]:
# 将所有时刻的所有点的位置信息存储到同一变量中，301行，223列，第一列为龙头，其余为龙身, 最后一列为龙尾后把手
theta_values_of_each_time = np.hstack((theta_head_value.reshape(-1, 1), theta_body_values_of_each_time))

# 计算每个点的 r 值
r_values = np.vectorize(compute_radius)(theta_values_of_each_time)

# 计算所有点的 x 和 y 坐标
x_values = r_values * np.cos(theta_values_of_each_time)
y_values = r_values * np.sin(theta_values_of_each_time)

# 创建多层列标签，分别表示 x 和 y 坐标
column_labels_x = [f'Point_{i}_x' for i in range(224)]
column_labels_y = [f'Point_{i}_y' for i in range(224)]
column_labels = column_labels_x + column_labels_y

# 合并 x 和 y 数据
combined_values = np.hstack((x_values, y_values))

# 创建 timestamp 列，从 0 到 300，并转换为二维列向量
timestamps = np.arange(301).reshape(-1, 1)

# 将 timestamp 列添加到坐标数据最左侧
combined_values_with_timestamp = np.hstack((timestamps, combined_values))

# 更新列标签，添加 'timestamp' 列
column_labels = ['timestamp'] + column_labels

# 创建 DataFrame
df_coordinates = pd.DataFrame(combined_values_with_timestamp, columns=column_labels)

# 保存到 CSV 文件
df_coordinates.to_csv('./Q1/dragon_coordinates_Q1.csv', index=False)

print("x 和 y 坐标以及时间戳已成功保存到 './Q1/dragon_coordinates_Q1.csv'")


x 和 y 坐标以及时间戳已成功保存到 './Q1/dragon_coordinates_Q1.csv'


### 速度信息

In [1077]:
# 创建列标签
column_labels_velocity = [f'Point_{i}_v' for i in range(224)] 

# 创建 timestamp 列
timestamps = np.arange(301)  # 从 0 到 300

# 将 timestamp 列添加到速度数据中
velocity_with_timestamp = np.column_stack((timestamps, velocity_body_values_of_each_time))


# 创建 DataFrame
df_velocity = pd.DataFrame(velocity_with_timestamp, columns=['timestamp'] + column_labels_velocity)

# 保存到 CSV 文件
df_velocity.to_csv('./Q1/dragon_velocity_Q1.csv', index=False)

print("速度信息已成功保存到 './Q1/dragon_velocity_Q1.csv'")


速度信息已成功保存到 './Q1/dragon_velocity_Q1.csv'


# 问题二

### 基本函数

#### 计算垂直于板凳的方向向量的单位向量 $\vec h_n$

In [1078]:
def compute_perpendicular_unit_vector(l_n):
    """
    计算垂直于已知向量 l_n 的单位向量 h_n
    输入:
        l_n - 已知的向量（二维或三维）
    输出:
        h_n - 垂直于 l_n 的单位向量
    """
    # 将输入转换为 numpy 数组
    l_n = np.array(l_n)

    # 交换分量并改变符号，得到一个垂直的向量
    h_n = np.array([-l_n[1], l_n[0]])

    # 归一化
    h_n = h_n / np.linalg.norm(h_n)
    
    return h_n

#### 计算初始点的四个顶点坐标

In [1079]:
def compute_head_vertices(θ,θ1):
    """
    计算龙头的四个顶点坐标
    输入:
        θ - 龙头的前把手的位置
        θ1 - 第一块龙身的前把手的位置
    输出:
        vertices_0 - 初始四个顶点的坐标列表
        l0 - 龙头的方向向量
        h0 - 垂直于 l0 的单位向量
    """
    # 计算龙头位置的极径
    r = compute_radius(θ)

    # 计算龙头的坐标
    x0 = r * np.cos(θ)
    y0 = r * np.sin(θ)

    # 计算垂直于 龙头的方向向量, θ 为龙头的前把手的位置，θ1 为第一块龙身的前把手的位置
    l_0_x, l_0_y= compute_bench_direction_vector(θ, θ1)
    l0 = np.array(compute_unit_vector(l_0_x, l_0_y))

    # 计算垂直于 l0 的单位向量 h0
    h0 = compute_perpendicular_unit_vector(l0)
    
    vertices_0 = [
        np.array([x0, y0]) + 0.15 * h0 + 0.275 * l0,  # 顶点 a1_0
        np.array([x0, y0]) + 0.15 * h0 - 3.135 * l0,  # 顶点 a2_0
        np.array([x0, y0]) - 0.15 * h0 + 0.275 * l0,  # 顶点 a3_0
        np.array([x0, y0]) - 0.15 * h0 - 3.135 * l0,  # 顶点 a4_0
    ]
    return vertices_0, l0, h0

#### 计算当前点的四个顶点坐标

In [1080]:
def compute_body_vertices(θ,θ1):
    """
    计算当前龙身的四个顶点坐标
    输入:
        θ - 当前的龙身的前把手的位置
        θ1 - 后一个龙身的前把手的位置
    输出:
        vertices_n - 当前时间点的四个顶点坐标列表
        l_n - 当前向量
        h_n - 当前垂直单位向量
    """

    # 计算当前龙身位置的极径
    r = compute_radius(θ)

    # 计算当前龙身的坐标
    x_n = r * np.cos(θ)
    y_n = r * np.sin(θ)

    # 计算垂直于 龙头的方向向量, θ 为龙头的前把手的位置，θ1 为第一块龙身的前把手的位置
    
    l_n_x, l_n_y= compute_bench_direction_vector(θ, θ1)
    l_n = np.array(compute_unit_vector(l_n_x, l_n_y))

    # 计算垂直于 l0 的单位向量 h0
    h_n = compute_perpendicular_unit_vector(l_n)
    
    # 四个顶点的坐标列表
    vertices_n = [
        np.array([x_n, y_n]) + 0.15 * h_n + 0.275 * l_n,  # 顶点 a1_n
        np.array([x_n, y_n]) + 0.15 * h_n - 1.925 * l_n,  # 顶点 a2_n
        np.array([x_n, y_n]) - 0.15 * h_n + 0.275 * l_n,  # 顶点 a3_n
        np.array([x_n, y_n]) - 0.15 * h_n - 1.925 * l_n,  # 顶点 a4_n
    ]
    return vertices_n, l_n, h_n

### 实现分离轴定理 SAT

In [1081]:
def projection_interval(vertices, axis):
    """
    计算给定轴上的投影区间。

    参数:
        vertices (list of np.array): 顶点坐标列表。每个顶点是一个 numpy 数组。
        axis (np.array): 投影轴，是一个 numpy 数组。

    返回:
        tuple: 投影区间 (最小值, 最大值)。
    """
    # 计算每个顶点在投影轴上的投影
    projections = [np.dot(vertex, axis) for vertex in vertices]
    
    # 计算投影的最小值和最大值
    min_projection = min(projections)
    max_projection = max(projections)
    
    return min_projection, max_projection

def rectangles_intersect(vertices1, vertices2):
    """
    使用分离轴定理检查两个矩形是否相交。
    参数:
        vertices1 (list of np.array): 第一个矩形的顶点坐标列表。
        vertices2 (list of np.array): 第二个矩形的顶点坐标列表。
    返回:
        bool: 如果两个矩形相交，则返回 True,否则返回 False。
    """
    # print(f"测试 矩阵1: {vertices1} 和 \n 矩阵2: {vertices2}")
    # 获取矩形的两条边（注意：矩形的边是相邻顶点之差）
    edges1 = [vertices1[i] - vertices1[(i + 1) % len(vertices1)] for i in range(2)]  # 只需要前两条边
    edges2 = [vertices2[i] - vertices2[(i + 1) % len(vertices2)] for i in range(2)]

    # 计算所有边的法线向量作为分离轴
    axes = [compute_perpendicular_unit_vector(edge) for edge in edges1 + edges2]

    # 对每个分离轴进行投影并检查是否有重叠
    for axis in axes:
        min1, max1 = projection_interval(vertices1, axis)
        min2, max2 = projection_interval(vertices2, axis)
        if max1 < min2 or max2 < min1:
            return False  # 没有相交

    return True  # 相交

## 分离轴定理 计算是否发生碰撞
只考虑前13个板子（包括龙头）

### 待使用的变量

In [1082]:
theta_values_of_each_time # 每个时间点所有点的 theta 值，龙头序号为 0，龙尾后把手序号为 223，(301, 224)

array([[ 0.00000000e+00,  3.26054808e-01,  5.13260886e-01, ...,
         3.67751728e+01,  3.69236240e+01,  3.70719503e+01],
       [-1.57785032e-01,  1.68641444e-01,  3.56058860e-01, ...,
         3.66504040e+01,  3.67989605e+01,  3.69473917e+01],
       [-3.15818466e-01,  1.09811165e-02,  1.98610680e-01, ...,
         3.65254920e+01,  3.66741541e+01,  3.68226906e+01],
       ...,
       [-7.48235809e+01, -7.41114703e+01, -7.37099890e+01, ...,
        -1.37150388e+01, -1.35068952e+01, -1.32990961e+01],
       [-7.54472470e+01, -7.47279709e+01, -7.43226395e+01, ...,
        -1.40328087e+01, -1.38241368e+01, -1.36158122e+01],
       [-7.60867942e+01, -7.53600159e+01, -7.49506597e+01, ...,
        -1.43562508e+01, -1.41470385e+01, -1.39381761e+01]])

### 计算前13个点前1000秒的位置坐标

In [1083]:
# 设定p的值
global p 
p = 1.7


# 计算需要判断 n 个点，是否发生碰撞
n = np.ceil( ( 4.45 + 3 * p ) * 2 * np.pi / 1.65 )
n =2* int(n) # 整数化n
print(f"需要计算n={n}个点") 

# 计算临界时间
def Cut_off_point(p,x):
    Cut_off_point = ( p / (2 * np.pi) * (f(9 * np.pi / p + 6* np.pi) - f( x + 9 * np.pi / p + 6 * np.pi)) )
    return Cut_off_point

# 用以调整碰撞的临界时间 g(-6π)
cut_off_point = Cut_off_point(p,-6*np.pi)
print(cut_off_point)

需要计算n=74个点
132.99182261140473


In [1084]:
# 计算龙头的所有坐标

# 初始化时间数组
time_array = np.arange(1000)  # 从 0 到 t 的整数数组

# 计算每个时间点的角度值（弧度制）
theta_head_value_2 = np.array([solve_θ_of_dragonhead(t) for t in time_array])

# 计算每个时间点的龙头坐标
length_of_dragon_head = 2.86 # 龙头的长度 286cm=2.86m
distance_of_dragon_body= 1.65 # 龙身上两个孔的距离 165cm=1.65m

# 创建二维数组来存储每个时间点的所有龙身的 θ 值
theta_body_values_of_each_time_2 = np.empty((1000, n))  # 301 行，222 列；301个时间点，221个龙身，1个龙尾

# 计算每个时间点的所有龙身的 θ 值
for t in range(1000):
    temp =  theta_head_value_2[t] # 当前时间内点的龙头对应的θ值
    for i in range(n):
        if i == 0:
            theta_body_values_of_each_time_2[t, i] = compute_theta_next(temp, length_of_dragon_head) # 当前时间点的龙身每个点的θ值
            temp = theta_body_values_of_each_time_2[t, i] # 更新当前时间点的龙身对应的θ值
        else:
            theta_body_values_of_each_time_2[t, i] = compute_theta_next(temp, distance_of_dragon_body) # 当前时间点的龙身每个点的θ值
            temp = theta_body_values_of_each_time_2[t, i] # 更新当前时间点的龙身对应的θ值


# 将所有时刻的所有点的位置信息存储到同一变量中，301行，13列，第一列为龙头，其余为龙身
theta_values_of_each_time_2 = np.hstack((theta_head_value_2.reshape(-1, 1), theta_body_values_of_each_time_2))

# 创建DataFrame并添加列名
columns = ['Head'] + [f'Body_{i+1}' for i in range(n)]
df = pd.DataFrame(theta_values_of_each_time_2, columns=columns)

# 保存为CSV文件
df.to_csv('./Q3/theta_values_of_each_time_Q3.csv', index=False)

print("CSV文件已成功保存！")

d:\conda_envs\ISLP\lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
d:\conda_envs\ISLP\lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


CSV文件已成功保存！


### 情况一：龙头撞龙身

In [1085]:
t=-1 # 从 t，每一轮结束，t=t+1，直到发生碰撞，弹出循环，返回发生碰撞时的t的值

while True:
    t = t + 1 # 每一轮结束，t=t+1

    # 计算得到当前时刻的各块的的位置所对应的 θ 值
    
    # 计算得到龙头的相关参数
    θ_0 = theta_values_of_each_time_2[t,0] # 龙头的位置所对应的 theta 值
    θ_1 = theta_values_of_each_time_2[t,1] # 龙头下一个位置（即第一块龙身）所对应的 theta 值

    # 计算得到的四个顶点坐标 和 龙头的方向向量 和 与龙头的方向向量垂直的单位向量 h_0
    vertices_0, l_0, h_0 = compute_head_vertices(θ_0, θ_1)

    for i in range(2, n): # 遍历前12个龙身(加上龙头，共13个)，排除相邻的块
        θ_n = theta_values_of_each_time_2[t,i] # 第 i 块龙身所对应的 theta 值
        θ_n1 = theta_values_of_each_time_2[t,i+1] # 第 i+1 块龙身所对应的 theta 值

        # 计算得到的四个顶点坐标 和 龙身的方向向量 和 与龙身方向向量垂直的单位向量 h_n
        vertices_n, l_n, h_n = compute_body_vertices(θ_n, θ_n1)

        # 判断是否发生碰撞
        if rectangles_intersect(vertices_0, vertices_n):
            print(f"碰撞发生，t = {t}, 龙头与 块编号为 {i} 发生碰撞,p={p}" )
            
            break  # 发生碰撞时跳出 for 循环并终止 while 循环
    else:
        # 如果没有碰撞，继续下一个时间点
        continue
    
    # 跳出 while 循环
    break


碰撞发生，t = 702, 龙头与 块编号为 2 发生碰撞,p=1.7


### 情况二：龙身撞龙身

In [1086]:
t=-1 # 从 t=300s 开始，每一轮结束，t=t+1，直到发生碰撞，弹出循环，返回发生碰撞时的t的值
flag = 0 # 初始化标志位
while True:
    t = t + 1 # 每一轮结束，t=t+1

    for i in range(1, n): # 遍历前12个龙身(加上龙头，共13个)，排除相邻的块
        θ_i = theta_values_of_each_time_2[t,i] # 第 i 块龙身所对应的 theta 值
        θ_i1 = theta_values_of_each_time_2[t,i+1] # 第 i+1 块龙身所对应的 theta 值

        # 计算得到的四个顶点坐标 和 龙身的方向向量 和 与龙身方向向量垂直的单位向量 h_n
        vertices_i, l_i, h_i = compute_body_vertices(θ_n, θ_n1)

        for j in range(i+2, n): # 遍历前12个龙身(加上龙头，共13个)，排除相邻的块
            # 如果j >=13，则跳出循环,到达边界
            if j >=n:
                break
            θ_j = theta_values_of_each_time_2[t,j] # 第 i 块龙身所对应的 theta 值
            θ_j1 = theta_values_of_each_time_2[t,j+1] # 第 i+1 块龙身所对应的 theta 值

            # 计算得到的四个顶点坐标 和 龙身的方向向量 和 与龙身方向向量垂直的单位向量 h_n
            vertices_j, l_j, h_j = compute_body_vertices(θ_j, θ_j1)

            # 判断是否发生碰撞
            if rectangles_intersect(vertices_i, vertices_j):
                print(f"碰撞发生，t = {t}, 块编号: {i} 和 {j},p={p}")
                flag = 1    # 标志位设为1，表示发生碰撞
        
        if flag == 1: # 如果发生碰撞，跳出 for 循环
            break

    if flag == 1: # 如果发生碰撞，跳出 while 循环
        break



碰撞发生，t = 0, 块编号: 1 和 3,p=1.7
碰撞发生，t = 0, 块编号: 1 和 4,p=1.7
碰撞发生，t = 0, 块编号: 1 和 5,p=1.7
碰撞发生，t = 0, 块编号: 1 和 6,p=1.7
碰撞发生，t = 0, 块编号: 1 和 7,p=1.7
碰撞发生，t = 0, 块编号: 1 和 8,p=1.7
碰撞发生，t = 0, 块编号: 1 和 9,p=1.7
碰撞发生，t = 0, 块编号: 1 和 10,p=1.7
碰撞发生，t = 0, 块编号: 1 和 11,p=1.7
碰撞发生，t = 0, 块编号: 1 和 12,p=1.7
碰撞发生，t = 0, 块编号: 1 和 13,p=1.7
碰撞发生，t = 0, 块编号: 1 和 14,p=1.7
碰撞发生，t = 0, 块编号: 1 和 15,p=1.7
碰撞发生，t = 0, 块编号: 1 和 16,p=1.7
碰撞发生，t = 0, 块编号: 1 和 17,p=1.7
碰撞发生，t = 0, 块编号: 1 和 18,p=1.7
碰撞发生，t = 0, 块编号: 1 和 19,p=1.7
碰撞发生，t = 0, 块编号: 1 和 20,p=1.7
碰撞发生，t = 0, 块编号: 1 和 21,p=1.7
碰撞发生，t = 0, 块编号: 1 和 22,p=1.7
碰撞发生，t = 0, 块编号: 1 和 23,p=1.7
碰撞发生，t = 0, 块编号: 1 和 24,p=1.7
碰撞发生，t = 0, 块编号: 1 和 25,p=1.7
碰撞发生，t = 0, 块编号: 1 和 26,p=1.7
碰撞发生，t = 0, 块编号: 1 和 27,p=1.7
碰撞发生，t = 0, 块编号: 1 和 28,p=1.7
碰撞发生，t = 0, 块编号: 1 和 29,p=1.7
碰撞发生，t = 0, 块编号: 1 和 30,p=1.7
碰撞发生，t = 0, 块编号: 1 和 31,p=1.7
碰撞发生，t = 0, 块编号: 1 和 32,p=1.7
碰撞发生，t = 0, 块编号: 1 和 33,p=1.7
碰撞发生，t = 0, 块编号: 1 和 34,p=1.7
碰撞发生，t = 0, 块编号: 1 和 35,p=1.7
碰撞发生，t = 0, 块编号: 

### 计算并导出碰撞前一刻舞龙队的位置和速度
计算得出，当舞龙队发生碰撞时间为`402s`，`第1块`龙身和`第5块`龙身发生碰撞。
故，为使舞龙队不发生碰撞，应在`402s`时停止盘入，下面导出此时舞龙队的位置和速度。

#### 位置信息

计算出 `t=402s` 时，舞龙队的位置信息，用 $\theta$ 表示

In [1087]:
# 402s时的龙头前把手位置
θ0_402 = theta_values_of_each_time_2[402,0] # 龙头所对应的 theta 值

# 板凳龙的基本参数
length_of_dragon_head = 2.86 # 龙头的长度 286cm=2.86m
distance_of_dragon_body= 1.65 # 龙身上两个孔的距离 165cm=1.65m

# 创建列表存储该时间点的所有龙身的 θ 值
theta_body_values_of_402 = np.empty(223)  # 222 个龙身，1个龙尾后把手

# 计算得到此时所有龙身的 θ 值
temp = θ0_402
for i in range(223):
    
    if i == 0:
        theta_body_values_of_402[i] = compute_theta_next(temp, length_of_dragon_head) # 当前时间点的龙身每个点的θ值
        temp = theta_body_values_of_402[i] # 更新当前时间点的龙身对应的θ值
    else:
        theta_body_values_of_402[i] = compute_theta_next(temp, distance_of_dragon_body) # 当前时间点的龙身每个点的θ值
        temp = theta_body_values_of_402[i] # 更新当前时间点的龙身对应的θ值

# 合并在 t=402s 时，龙头和龙尾的位置信息
theta_values_of_402 =  np.hstack((θ0_402, theta_body_values_of_402))


将极坐标转换为直角坐标

In [1088]:
# 计算每个点的 r 值
r_values = np.vectorize(compute_radius)(theta_values_of_402)

# 计算所有点的 x 和 y 坐标
x_values = r_values * np.cos(theta_values_of_402)
y_values = r_values * np.sin(theta_values_of_402)


# 初始化坐标类型标签和对应的数据列表
labels = []
values = []

# 填充标签和数据，交替放入 x 和 y 值
for i in range(0, 224):  # i 从 1 到 224
    labels.append(f"Point_{i}_x")  # 添加 x 标签
    labels.append(f"Point_{i}_y")  # 添加 y 标签
    values.append(x_values[i])  # 添加对应的 x 值
    values.append(y_values[i])  # 添加对应的 y 值

labels = np.array(labels).flatten()

# 确保 labels 和 values 都是一维列表，且长度匹配
# labels = np.array(labels)  # 转换为一维数组
# values = np.array(values)  # 转换为一维数组

# 确保 labels 和 values 长度相同
print(f"Length of labels: {len(labels)}, Length of values: {len(values)}")

# 创建 DataFrame
df_coordinates = pd.DataFrame({
    "Type": labels,          # 第一列为坐标类型
    "Value": values        # 第二列为对应的 x 或 y 数据
})

# 保存到 CSV 文件
df_coordinates.to_csv('./Q2/dragon_coordinates_Q2.csv', index=False)

print(df_coordinates.head())  # 打印前几行查看结果



Length of labels: 448, Length of values: 448
        Type     Value
0  Point_0_x -4.178665
1  Point_0_y  1.542624
2  Point_1_x -4.439416
3  Point_1_y -1.305465
4  Point_2_x -3.787931


#### 速度

In [1089]:
# 建立一个空的数组来存储每个时间点龙身每个点的速度
velocity_body_values_of_each_time = np.empty((301, 224))  # 301 行，223 列；301个时间点，一个龙头，221个龙身，1个龙尾，最后一列为龙尾后把手

for t in range(301):
    temp = speed_of_head # 计算当前时间点的龙头速度大小
    velocity_body_values_of_each_time[t, 0] = temp # u0 存放龙头的速度大小
    for i in range(0,223):
        velocity_body_values_of_each_time[t, i+1] = compute_velocity_next(theta_values_of_each_time[t,i], theta_values_of_each_time[t,i+1], temp)
        temp = velocity_body_values_of_each_time[t, i+1]

In [1090]:
#  speed_of_head 表示每个时刻的龙头的速度大小
speed_of_head= 1

# 建立一个空的数组来存储312s每个点的速度
velocity_values_of_402 = np.empty(224)  # ，224 列；1个龙头，221个龙身，1个龙尾，最后一列为龙尾后把手

temp = speed_of_head
velocity_values_of_402[0] = temp
for i in range(0,223):
    velocity_values_of_402[i+1] = compute_velocity_next(theta_values_of_402[i], theta_values_of_402[i+1], temp)
    temp = velocity_values_of_402[i+1]

# 创建点的编号（从 0 开始到 223），共 224 个点
points = np.arange(224)

# 创建 DataFrame，将点编号和速度数据存储为两列
df_velocity = pd.DataFrame({
    "Point": points,  # 第一列为点的编号，范围从 0 到 223
    "Velocity": velocity_values_of_402  # 第二列为对应的速度值
})

# 保存 DataFrame 到 CSV 文件
df_velocity.to_csv('./Q2/dragon_velocity_values_Q2.csv', index=False)

print("CSV 文件已成功保存！")

CSV 文件已成功保存！
